# Libraries

In [9]:
import pandas as pd
import numpy as np

from ethanol_brazil_plants.utils import utils

from datetime import date
from xlrd.xldate import xldate_as_datetime

pd.options.display.max_columns = 100

# Functions

In [10]:
def normalize_renovabio_database(
    raw_df: pd.DataFrame,
    object_cols: list[str],
    int_cols: list[str],
    float_cols: list[str],
    date_cols: list[str],
) -> pd.DataFrame:
    """
    Normalize renovabio database columns
    """

    normalized_df = utils.normalize_header(raw_df)
    normalized_df = normalized_df.fillna(method='ffill')

    normalized_df = utils.normalize_object_columns(normalized_df, object_cols)
    normalized_df = normalized_df[normalized_df['CNPJ'] != '']

    normalized_df = utils.normalize_int_columns(normalized_df, int_cols)
    normalized_df = utils.normalize_float_columns(normalized_df, float_cols)

    normalized_df = utils.normalize_date_columns(normalized_df, date_cols)

    return normalized_df

In [11]:
def split_razao_cidade_uf_column(normalized_df: pd.DataFrame) -> pd.DataFrame:
    """
    Split RAZAO_SOCIAL-CIDADE-UF column into RAZAO_SOCIAL, CIDADE and UF columns.
    """

    normalized_df['RAZAO_SOCIAL'] = normalized_df['RAZAO_SOCIAL-CIDADE-UF'].str.split('-').str[0]

    normalized_df[['CIDADE', 'UF', 'UF_2']] = (
        normalized_df['RAZAO_SOCIAL-CIDADE-UF']
        .str.split('-')
        .str[-2:]
        .str.join(',')
        .str.split(',', expand=True)
    )

    normalized_df['UF'] = np.where(
        normalized_df['UF'].isnull(),
        normalized_df['UF_2'],
        normalized_df['UF']
    )

    normalized_df['CIDADE'] = np.where(
        normalized_df['RAZAO_SOCIAL'] !=
        normalized_df['CIDADE'],
        normalized_df['CIDADE'],
        None
    )

    normalized_df = normalized_df.drop(['RAZAO_SOCIAL-CIDADE-UF', 'UF_2'], axis=1)
    return normalized_df

In [12]:
def abbreviate_production_route(
    normalized_df: pd.DataFrame,
    ds_rota: dict,
    cd_rota: dict
) -> pd.DataFrame:
    """
    Abbreviate routes names and define codes for each one of them.
    Args:
        normalized_df: Normalized dataframe.
        ds_rota: Dictionary with routes names as keys and abbreviations as values.
        cd_rota: Dictionary with routes abbreviations as keys and codes as values.
    """

    normalized_df['DS_ROTA'] = normalized_df['ROTA'].replace(ds_rota)
    normalized_df['CD_ROTA'] = normalized_df['DS_ROTA'].replace(cd_rota)
    return normalized_df

In [13]:
def generate_zip_code(
    normalized_df: pd.DataFrame,
    address_column: str = 'ENDERECO_EMISSOR_PRIMARIO'
) -> pd.Series:
    """
    Extract zip code (CEP) from address.
    Args:
        normalized_df: DataFrame with normalized data;
        address_column: Column with address.
    """

    return (
        normalized_df[address_column]
        .str.replace('.','')
        .str.replace(r'-(\s+)', '-', regex=True)
        .str.findall(r'\d{5}-\d{3}|\d{5}-\d{2}')
        .str[0]
        .str.replace('-','')
        .str.pad(8, side='right', fillchar='0')
    )

In [14]:
def generate_refined_address(
    normalized_df: pd.DataFrame,
    address_column: str = "ENDERECO_EMISSOR_PRIMARIO"
) -> pd.Series:
    """
    Generate a refined address column from address column.
    Args:
        normalized_df: Normalized DataFrame;
        address_column: Address column name.
    """
    return (
        normalized_rbp_validos['ENDERECO_EMISSOR_PRIMARIO']
        .str.replace(r'S/N[\w\W]| NO ', '', regex= True)
        .str.replace(r'CEP[\W]', '', regex=True)
        .str.replace('.', '')
        .str.replace(', ,', ',')
        .str.replace(r'\s+', ' ', regex= True)
    )

In [15]:
def extract_city_and_state(
    normalized_df: pd.DataFrame,
    refined_address_col: str = 'DS_END',
) -> pd.DataFrame:
    """
    Extracts city (CIDADE) and state (UF) from a normalized address column.

    Args:
        normalized_df: DataFrame with a refined address column.
        refined_address_col: Name of refined address column.
    """

    normalized_df[['re_CIDADE', 're_UF']] = (
        normalized_df['DS_END']
        .str.extract(r'([A-Z\s]+)(?: - |/| / )([A-Z]{2})\b')
    )

    for col in ['CIDADE', 'UF']:
        normalized_df[col] = (
            normalized_df[col]
            .fillna(normalized_df[f're_{col}'])
        )
        normalized_df = normalized_df.drop(f're_{col}', axis=1)
    
    return normalized_df

In [16]:
def split_address_column(
    normalized_df: pd.DataFrame,
    refined_address_column: str = "DS_END",
) -> pd.DataFrame:
    """
    Split address column into main address and number columns.
    Args:
        normalized_df: Normalized dataframe.
        refined_address_column: Column name of refined address column.
    """
    
    # Pattern 1 - Highway and Roadway
    normalized_df['p1_DS_END'] = (
        normalized_df['DS_END']
        .str.extract(r'(^RO[\w\s]+|[\s\W][A-Z]{2}(?<!KM|LT|CP)[\s\W]\d{1,3}\s|^EST[\w\s]+|ESTRADA[\w\s]+)')
    )

    normalized_df['p1_NO_END'] = (
        normalized_df['DS_END']
        .str.extract(r'(KM[\s\W]+\d+)')
    )

    # Pattern 2 - Farm
    normalized_df[['p2_DS_END']] = (
        normalized_df['DS_END']
        .str.extract(r'(^FAZ[\w|\s]+)')
    )

    normalized_df[['p2_NO_END']] = 'SN'

    # Pattern 3 - Street
    normalized_df[['p3_DS_END', 'p3_NO_END']] = (
        normalized_df['DS_END']
        .str.extract(r'(R[\s][A-Z\s]+|RUA[\w\s]+)(?:, |[\W]|,)([\d]+)')
    )

    # Pattern 4 - Avenue
    normalized_df['p4_DS_END'] = (
        normalized_df['DS_END']
        .str.extract(r'(^AV[\w\s]+|^AV[\s\w]+)')
    )

    normalized_df['p4_NO_END'] = (
        normalized_df['DS_END']
        .str.extract(r'(?:^AV[\w\s]+|^AV[\s\w]+)(?:, |[\W]|,)([\d]+)')
    )
    # Pattern 5 - Others
    normalized_df['p5_DS_END'] = (
        normalized_df['DS_END']
        .str.split(',')
        .str[0]
    )

    normalized_df['p5_NO_END'] = (
        normalized_df['DS_END']
        .str.split(',')
        .str[1]
    )

    # Addresses
    address_cols = ['DS_END', 'NO_END']
    for col in address_cols:
        normalized_df[f'{col}'] = (
            normalized_df[f'p1_{col}']
            .fillna(normalized_df[f'p2_{col}'])
            .fillna(normalized_df[f'p3_{col}'])
            .fillna(normalized_df[f'p4_{col}'])
            .fillna(normalized_df[f'p5_{col}'])
        )
        normalized_df = normalized_df.drop(
            [f'p1_{col}', f'p2_{col}', f'p3_{col}',
            f'p4_{col}', f'p5_{col}'], axis=1)

    return normalized_df

In [17]:
def rename_and_reorder_column(
    df: pd.DataFrame,
    dict_rename_cols: dict,
    ordered_cols: list,
) -> pd.DataFrame:
    """
    Rename and reorder columns of DataFrame.
    Args:
        df: DataFrame to be renamed and reordered.
        dict_rename_cols: Dictionary with old and new column names.
        ordered_cols: List with ordered columns.
    """

    df = df.rename(dict_rename_cols, axis=1)
    df = df[ordered_cols].copy()
    return df

# Data

In [18]:
raw_rbp_validos = catalog.load("raw_renovabio_plants_validos")
raw_rbp_canc_sup = catalog.load("raw_renovabio_plants_canc_susp")
raw_rbp_anulados = catalog.load("raw_renovabio_plants_anulados")

[08/08/23 16:00:39] INFO     Loading data from 'raw_renovabio_plants_validos' (ExcelDataSet)... ]8;id=191355;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=774493;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'raw_renovabio_plants_canc_susp'                 ]8;id=715847;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=996744;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\
                             (ExcelDataSet)...                                                                     

                    INFO     Loading data from 'raw_renovabio_plants_anulados'                  ]8;id=124748;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=285242;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\
                             (ExcelDataSet)...                                                                     

# Normalize

In [19]:
normalized_rbp_validos = normalize_renovabio_database(
    raw_rbp_validos,
    catalog.load('params:object_cols'),
    catalog.load('params:int_cols'),
    catalog.load('params:float_cols'),
    catalog.load('params:date_cols'),
)

                    INFO     Loading data from 'params:object_cols' (MemoryDataset)...          ]8;id=861391;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=450859;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:int_cols' (MemoryDataset)...             ]8;id=368340;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=584185;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:float_cols' (MemoryDataset)...           ]8;id=220839;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=503860;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:date_cols' (MemoryDataset)...            ]8;id=75892;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=713265;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

# Business name, City and State

In [20]:
normalized_rbp_validos = split_razao_cidade_uf_column(normalized_rbp_validos)

# Route

In [21]:
normalized_rbp_validos = abbreviate_production_route(
    normalized_rbp_validos,
    catalog.load('params:ds_rota'),
    catalog.load('params:cd_rota')
)

                    INFO     Loading data from 'params:ds_rota' (MemoryDataset)...              ]8;id=996295;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=129158;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:cd_rota' (MemoryDataset)...              ]8;id=200960;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=807720;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

# Location

## ZIP Code

In [22]:
normalized_rbp_validos['CEP'] = generate_zip_code(normalized_rbp_validos)

                    WARNING  C:\Users\daniel.silva\AppData\Local\Temp\ipykernel_18804\896430648.py: ]8;id=868583;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\warnings.py\warnings.py]8;;\:]8;id=235057;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\warnings.py#109\109]8;;\
                             13: FutureWarning: The default value of regex will change from True to                
                             False in a future version. In addition, single character regular                      
                             expressions will *not* be treated as literal strings when regex=True.                 
                               normalized_df[address_column]                                                       
                                                                                                                   

## City and State

In [23]:
normalized_rbp_validos['DS_END'] = generate_refined_address(normalized_rbp_validos)
normalized_rbp_validos = extract_city_and_state(normalized_rbp_validos)


[08/08/23 16:00:40] WARNING  C:\Users\daniel.silva\AppData\Local\Temp\ipykernel_18804\1426024363.py ]8;id=694454;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\warnings.py\warnings.py]8;;\:]8;id=812002;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\warnings.py#109\109]8;;\
                             :12: FutureWarning: The default value of regex will change from True                  
                             to False in a future version. In addition, single character regular                   
                             expressions will *not* be treated as literal strings when regex=True.                 
                               normalized_rbp_validos['ENDERECO_EMISSOR_PRIMARIO']                                 
                                                                                                                   

## Address

In [24]:
normalized_rbp_validos = split_address_column(normalized_rbp_validos)

# Renaming and ordering columns

In [28]:
normalized_rbp_validos = rename_and_reorder_column(
    normalized_rbp_validos,
    catalog.load('params:dict_rename_cols'),
    catalog.load('params:ordered_cols')
)

[08/08/23 16:00:57] INFO     Loading data from 'params:dict_rename_cols' (MemoryDataset)...     ]8;id=641237;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=480091;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:ordered_cols' (MemoryDataset)...         ]8;id=438863;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=530651;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

In [29]:
def generate_refined_renovabio_database(
    raw_renovabio_database: pd.DataFrame,
    object_cols: list,
    int_cols: list,
    float_cols: list,
    date_cols: list,
    ds_rota: dict,
    cd_rota: dict,
    dict_rename_cols: dict,
    ordered_cols: list,
) -> pd.DataFrame:
    """
    Generate a refined Renovabio database.
    """

    # Normalize
    normalized_database = normalize_renovabio_database(
        raw_renovabio_database,
        object_cols,
        int_cols,
        float_cols,
        date_cols,
    )

    # Business name, City and State
    normalized_database = split_razao_cidade_uf_column(normalized_database)

    # Route
    normalized_database = abbreviate_production_route(
        normalized_database, ds_rota, cd_rota
    )

    # Zip code
    normalized_database["CEP"] = generate_zip_code(normalized_database)
    # City and State
    normalized_database["DS_END"] = generate_refined_address(
        normalized_database
    )
    normalized_database = extract_city_and_state(normalized_database)
    # Address
    normalized_database = split_address_column(normalized_database)

    # Rename and reorder columns
    normalized_database = rename_and_reorder_column(
        normalized_database, dict_rename_cols, ordered_cols
    )

    return normalized_database

In [30]:
object_cols = catalog.load('params:object_cols')
int_cols = catalog.load('params:int_cols')
float_cols = catalog.load('params:float_cols')
date_cols = catalog.load('params:date_cols')
ds_rota = catalog.load('params:ds_rota')
cd_rota = catalog.load('params:cd_rota')
dict_rename_cols = catalog.load('params:dict_rename_cols')
ordered_cols = catalog.load('params:ordered_cols')

[08/08/23 16:01:00] INFO     Loading data from 'params:object_cols' (MemoryDataset)...          ]8;id=4581;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=221447;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:int_cols' (MemoryDataset)...             ]8;id=777465;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=157337;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:float_cols' (MemoryDataset)...           ]8;id=781957;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=35859;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:date_cols' (MemoryDataset)...            ]8;id=674722;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=74886;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:ds_rota' (MemoryDataset)...              ]8;id=499276;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=613968;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:cd_rota' (MemoryDataset)...              ]8;id=267424;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=19953;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:dict_rename_cols' (MemoryDataset)...     ]8;id=459372;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=673414;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

                    INFO     Loading data from 'params:ordered_cols' (MemoryDataset)...         ]8;id=522716;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=376895;file://c:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\